In [1]:
# 🧠 Chatbot sobre CVs en PDF con Pinecone + OpenAI/Groq
# =======================================================

In [2]:
## ✅ Paso 1: Instalación de librerías
!pip install -q pinecone openai PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.5 MB/s eta 0:00:00


In [3]:
## ✅ Paso 2: Importación de librerías y configuración
import fitz  # PyMuPDF
import pinecone
from openai import OpenAI
from google.colab import files

In [4]:
# Reemplazá con tus claves reales
PINECONE_API_KEY = ""
PINECONE_ENV = "gcp-starter"
OPENAI_API_KEY = ""

In [5]:
# Cliente OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
# Inicializar Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [7]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "cv-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',           # ✅ cambiar a AWS
            region='us-east-1'     # ✅ región habilitada en plan gratuito
        )
    )

index = pc.Index(index_name)


In [8]:
## ✅ Paso 3: Función para extraer texto de PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()

In [9]:
## ✅ Paso 4: Función para obtener embeddings
def get_embedding(texto):
    response = client.embeddings.create(
        input=[texto],
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [10]:
## ✅ Paso 5: Subir archivos PDF
print("📤 Subí los archivos PDF con los CVs...")
uploaded = files.upload()

📤 Subí los archivos PDF con los CVs...


Saving CV_VictoriaTeran.pdf to CV_VictoriaTeran.pdf


In [11]:
pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.6 MB/s eta 0:00:00


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from pdfminer.high_level import extract_text

# Paso 1: Cargar y extraer texto de un PDF
def extract_text_from_pdf(file_path):
    return extract_text(file_path)

# Paso 2: Crear objeto Document para LangChain
def create_document(text, metadata=None):
    if metadata is None:
        metadata = {}
    return Document(page_content=text, metadata=metadata)

# Paso 3: Aplicar chunking
def chunk_text(document, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents([document])

# --- USO DEL CÓDIGO ---

# Ruta al PDF
pdf_path = "CV_VictoriaTeran.pdf"  # reemplazar por tu path

# Extraer texto del PDF
full_text = extract_text_from_pdf(pdf_path)

# Crear objeto Document
doc = create_document(full_text, metadata={"source": pdf_path})

# Aplicar chunking
chunks = chunk_text(doc)

# Mostrar los chunks
for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk.page_content[:300])  # Mostramos solo los primeros 300 caracteres



--- Chunk 1 ---
Victoria Terán

DATA SCIENTIST

Contacto

Experiencia

mvictoriateran@gmail.com

Analista Sr de Riesgo de Crédito

11-3094-0685

in/mariavictoriateran

Habilidades
Técnicas

Python

SQL

Tableau

Qlikview

Office

Mercado Libre

2 0 2 2   -   A c t u a l i d a d

Desarrollo,  validación  y  monitore

--- Chunk 2 ---
Registros  contables  y  procesos  afines  a  la  concilicación

bancaria;  control  y  reposición  de  fondos  de  uso  interno;

participación en proyectos de automatización de procesos. 

Pasante

HSBC

2 0 1 5   -   2 0 1 5

Verificación, validación y registro de solicitudes de pólizas de

segur


In [13]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [14]:
from sentence_transformers import SentenceTransformer

# Cargamos un modelo local (rápido y bueno para recuperación semántica)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # o 'all-mpnet-base-v2'

def get_embedding(text):
    return embedding_model.encode(text).tolist()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
chunks[0]

Document(metadata={'source': 'CV_VictoriaTeran.pdf'}, page_content='Victoria Terán\n\nDATA SCIENTIST\n\nContacto\n\nExperiencia\n\nmvictoriateran@gmail.com\n\nAnalista Sr de Riesgo de Crédito\n\n11-3094-0685\n\nin/mariavictoriateran\n\nHabilidades\nTécnicas\n\nPython\n\nSQL\n\nTableau\n\nQlikview\n\nOffice\n\nMercado Libre\n\n2 0 2 2   -   A c t u a l i d a d\n\nDesarrollo,  validación  y  monitoreo  de  modelos  de  Machine\n\nLearning enfocados en la gestión del riesgo de crédito. \n\nDiseño  de  soluciones  analíticas  que  facilitan  la  toma  de\n\ndecisiones estratégicas en el negocio.\n\nAnalista de Datos\n\nAFIP\n\n2 0 1 9   -   2 0 2 2\n\nTrabajos  de  extracción,  medición  de  calidad,  análisis  y\n\ndefinición  de  variables  para  el  desarrollo  de  modelos\n\nanalíticos predictivos mediante el uso de grandes volúmenes\n\nde datos. \n\nAnalista administrativo Contable\n\nAFIP\n\n2 0 1 5   -   2 0 1 9\n\nRegistros  contables  y  procesos  afines  a  la  concilicación\n\nb

In [16]:
type(chunks)

list

In [17]:
type(chunks[0])

langchain_core.documents.base.Document

In [18]:
text_chunks = [doc.page_content for doc in chunks]

In [19]:
# Convertimos todos los chunks a vectores
vectors = []
for i, chunk in enumerate(text_chunks):
    vector = {
        "id": f"chunk-{i}",
        "values": get_embedding(chunk),
        "metadata": {
            "text": chunk
        }
    }
vectors.append(vector)

In [20]:
## ✅ Paso 6: Hacer una pregunta
pregunta = "¿Quién tiene experiencia en ciencia de datos y Python?"
embedding_pregunta = get_embedding(pregunta)

In [21]:
resultados = index.query(vector=embedding_pregunta, top_k=1, include_metadata=False)
id_mas_cercano = resultados['matches'][0]['id']
print("CV más cercano a la pregunta:", id_mas_cercano)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 12 Aug 2025 19:15:49 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '55', 'x-pinecone-request-id': '394398795739643333', 'x-envoy-upstream-service-time': '55', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 1536","details":[]}


In [ ]:
## ✅ Paso 7: Chatbot simple sobre el CV seleccionado
# Volvemos a extraer el texto del PDF relevante
pdf_filename = id_mas_cercano + ".pdf"
contexto = extract_text_from_pdf(pdf_filename)

In [ ]:
prompt = f"""
Actuá como un asistente profesional de RRHH. Basado en este CV:

In [ ]:
{contexto}

In [ ]:
Respondé la siguiente pregunta del usuario:
{pregunta}
"""

In [ ]:
respuesta = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

In [ ]:
print("Respuesta del chatbot:\n", respuesta.choices[0].message.content)
